In [1]:
from tqdm.notebook import tqdm
from skimage import measure
import numpy as np
import cv2
import os

def random_function():
    #for img_id in tqdm(range(9112)):
    for img_id in tqdm(os.listdir("Corrosao/labels.old/")):
        label_path = f"Corrosao/labels.old/{img_id}"
        img_path = f"Corrosao/imgs/{img_id}"
        img = cv2.imread(img_path)
        label = cv2.imread(label_path)
        if label is None or img is None:
            if label is None:
                tqdm.write(f"{label_path} nao carregado corretamente")
            if img is None:
                tqdm.write(f"{img_path} nao carregado corretamente")
            
            try:
                os.remove(label_path)
            except:
                pass
            try:
                os.remove(f"Corrosao/imgs/{img_id}")
            except:
                pass
        else:
            #print(label)
            numbers_list = np.unique(label)
            for number in numbers_list:
                if number > 1:
                    print(numbers_list)
                    return
            #for row in label:
            #    for pixel in row:
            #        if (pixel.all() != 0) and (pixel.all() != 1):
            #            print(pixel)
            #            return

            #print(measure.find_contours(label))
#random_function()

In [18]:
os.makedirs("Corrosao/labels/",exist_ok=True)
for img_id in tqdm(os.listdir("Corrosao/labels.old/")):
    label_path = f"Corrosao/labels.old/{img_id}"
    label = cv2.imread(label_path)
    if label is None:
        if label is None:
            tqdm.write(f"{label_path} nao carregado corretamente")
    else:
        #print(label)
        gray = cv2.cvtColor(label, cv2.COLOR_BGR2GRAY) # ADD THIS
        img = cv2.resize(gray, (label.shape[0],label.shape[1]))
        cv2.imwrite(f"Corrosao/labels/{img_id}",img)
        
                

FileNotFoundError: [WinError 3] O sistema não pode encontrar o caminho especificado: 'Corrosao/labels.old/'

In [5]:
from tqdm.notebook import tqdm
from skimage import measure
import numpy as np
import cv2
import os

number_of_images = 0
max_number_of_images = 10
max_number_of_images = float("inf")
dataset_name = "Corrosao_full"


def is_in_csv(csv_path, file_name):
    with open(csv_path, "r") as list:
        lines = list.readlines()
        for line in lines:
            line = line.strip()
            if file_name in line:
                return True
    return False

os.makedirs(f"datasets/{dataset_name}/labels/val/",exist_ok=True)
os.makedirs(f"datasets/{dataset_name}/images/val/",exist_ok=True)
os.makedirs(f"datasets/{dataset_name}/labels/train/",exist_ok=True)
os.makedirs(f"datasets/{dataset_name}/images/train/",exist_ok=True)
os.makedirs(f"datasets/{dataset_name}/labels/test/",exist_ok=True)
os.makedirs(f"datasets/{dataset_name}/images/test/",exist_ok=True)

#"""
for img_id in tqdm(os.listdir("datasets/Corrosao/labels_full/")):

    is_in_train = is_in_csv("datasets/Corrosao/filenames_train.csv",img_id)
    if is_in_train:
        is_in_test = False
        target_folder = "val"
    else:
        is_in_test = is_in_csv("datasets/Corrosao/filenames_test.csv",img_id)
        if is_in_test:
            target_folder = "val"
        else:
            target_folder = "train"


    is_in_any_csv = is_in_train or is_in_test
    if img_id[-3:] == "png" and number_of_images < max_number_of_images:

        label_path = f"datasets/Corrosao/labels_full/{img_id}"
        label = cv2.imread(label_path, cv2.IMREAD_GRAYSCALE)
        img_path = f"datasets/Corrosao/images_full/{img_id}"
        img = cv2.imread(img_path)
        
        if label is None or img is None:
            if label is None:
                tqdm.write(f"{label_path} nao carregado corretamente")
        else:
            countours =  measure.find_contours(label)

            label_text = ""
            with open(f"datasets/{dataset_name}/labels/{target_folder}/{img_id[:-3]}txt", "w") as my_file:
                for countour in countours:
                    label_text = label_text + "0 "
                    for point in countour:
                        x = point[0]/label.shape[0]
                        y = point[1]/label.shape[1]
                        label_text = label_text + str(y) +" "
                        label_text = label_text + str(x) +" "
                    label_text = label_text[:-1]+ "\n"
                    my_file.write(label_text)
            cv2.imwrite(f"datasets/{dataset_name}/images/{target_folder}/{img_id}", img)
        number_of_images += 1
#"""

  0%|          | 0/8685 [00:00<?, ?it/s]

In [8]:
from ultralytics import YOLO

model = YOLO('runs/segment/train_export_large_from_scratch/weights/best.pt')  # load a pretrained model (recommended for training)
results = model('SISTEMAS_UFF/Images/cubemap_P74_fb48bf8c7a98b06384f1f93fcf027adb_posx.png', save = True, boxes=False, line_width=0, show=True)  # predict on an image
#results = model('datasets/Corrosao/images_full/2174.png', save = True)  # predict on an image

WARNING  Environment does not support cv2.imshow() or PIL Image.show()
OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


image 1/1 f:\Pedro\YOLOv8\Darwin_dados\SISTEMAS_UFF\Images\cubemap_P74_fb48bf8c7a98b06384f1f93fcf027adb_posx.png: 640x640 1 Escadas, 8 Estruturass, 31 Guarda Corpos, 1 Piso, 2 Suportess, 43.2ms
Speed: 0.0ms preprocess, 43.2ms inference, 6.7ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\predict9


In [4]:
from ultralytics import YOLO
model = YOLO('yolov8n-seg.pt')
model.val(data="coco.yaml", device=0)

Ultralytics YOLOv8.0.192  Python-3.8.10 torch-1.13.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3060, 12287MiB)
YOLOv8n-seg summary (fused): 195 layers, 3404320 parameters, 0 gradients, 12.6 GFLOPs
val: Scanning F:\Pedro\YOLOv8\Darwin_dados\datasets\coco\labels\val2017.cache... 4952 images, 48 backgrounds, 0 corrupt: 100%|██████████| 5000/5000 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 313/313 [23:53<00:00,  4.58s/it]
                   all       5000      36335      0.621      0.481      0.518      0.364      0.622      0.457      0.491      0.306
                person       5000      10777       0.74      0.681      0.744      0.515      0.741      0.651       0.71      0.401
               bicycle       5000        314      0.628      0.385      0.445      0.246      0.585      0.334       0.36      0.135
                   car       5000       1918       0.63      0.5

loading annotations into memory...
Done (t=0.73s)
creating index...
index created!
Loading and preparing results...
DONE (t=9.98s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=82.20s).
Accumulating evaluation results...
DONE (t=20.13s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.367
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.522
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.398
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.179
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.404
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.521
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.315
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.532
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDe

Results saved to runs\segment\val10


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77,
       78, 79])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001F9BB4186A0>
fitness: 0.703978213285255
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)', 'metrics/precision(M)', 'metrics/recall(M)', 'metrics/mAP50(M)', 'metrics/mAP50-95(M)']
maps: array([    0.91537,     0.38099,     0.65159,     0.69212,      1.1301,      1.1679,      1.2347,     0.55256,     0.34079,     0.38233,       1.181,      1.1751,     0.84956,     0.33291,     0.48968,      1.2719,      1.1063,    